# 6.5 循环神经网络的简洁实现

In [2]:
import time
import math
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F

import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = d2l.load_data_jay_lyrics()

print(torch.__version__, device)

1.5.0 cuda


## 6.5.1 定义模型

In [3]:
print(vocab_size)

1027


In [4]:
num_hiddens = 256
# rnn_layer = nn.LSTM(input_size=vocab_size, hidden_size=num_hiddens) # 已测试
rnn_layer = nn.RNN(input_size=vocab_size, hidden_size=num_hiddens)

In [5]:
num_steps = 35
batch_size = 2
state = None
X = torch.rand(num_steps, batch_size, vocab_size)
Y, state_new = rnn_layer(X, state)
print(Y.shape, len(state_new), state_new[0].shape)

torch.Size([35, 2, 256]) 1 torch.Size([2, 256])


In [6]:
# 本类已保存在d2lzh_pytorch包中方便以后使用
class RNNModel(nn.Module):
    def __init__(self, rnn_layer, vocab_size):
        super(RNNModel, self).__init__()
        self.rnn = rnn_layer
        self.hidden_size = rnn_layer.hidden_size * (2 if rnn_layer.bidirectional else 1) 
        self.vocab_size = vocab_size
        self.dense = nn.Linear(self.hidden_size, vocab_size)
        self.state = None

    def forward(self, inputs, state): # inputs: (batch, seq_len)
        # 获取one-hot向量表示
        X = d2l.to_onehot(inputs, vocab_size) # X是个list
        Y, self.state = self.rnn(torch.stack(X), state)
        # 全连接层会首先将Y的形状变成(num_steps * batch_size, num_hiddens)，它的输出
        # 形状为(num_steps * batch_size, vocab_size)
        output = self.dense(Y.view(-1, Y.shape[-1]))
        return output, self.state

## 6.5.2 训练模型

In [7]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def predict_rnn_pytorch(prefix, num_chars, model, vocab_size, device, idx_to_char,
                      char_to_idx):
    state = None
    output = [char_to_idx[prefix[0]]] # output会记录prefix加上输出
    for t in range(num_chars + len(prefix) - 1):
        X = torch.tensor([output[-1]], device=device).view(1, 1)
        if state is not None:
            if isinstance(state, tuple): # LSTM, state:(h, c)  
                state = (state[0].to(device), state[1].to(device))
            else:   
                state = state.to(device)
            
        (Y, state) = model(X, state)  # 前向计算不需要传入模型参数
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y.argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])

In [8]:
model = RNNModel(rnn_layer, vocab_size).to(device)
predict_rnn_pytorch('分开', 10, model, vocab_size, device, idx_to_char, char_to_idx)

'分开它荡荡荡荡荡荡荡荡荡'

In [28]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes):
    loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.to(device)
    state = None
    for epoch in range(num_epochs):
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = d2l.data_iter_consecutive(corpus_indices, batch_size, num_steps, device) # 相邻采样
        for X, Y in data_iter:
            if state is not None:
                # 使用detach函数从计算图分离隐藏状态, 这是为了
                # 使模型参数的梯度计算只依赖一次迭代读取的小批量序列(防止梯度计算开销太大)
                if isinstance (state, tuple): # LSTM, state:(h, c)  
                    state = (state[0].detach(), state[1].detach())
                else:   
                    state = state.detach()
    
            (output, state) = model(X, state) # output: 形状为(num_steps * batch_size, vocab_size)
            
            # Y的形状是(batch_size, num_steps)，转置后再变成长度为
            # Messier:Y的每一行对应一个one_hot向量 y每一行的数值即是标签索引值
            # batch * num_steps 的向量，这样跟输出的行一一对应
            y = torch.transpose(Y, 0, 1).contiguous().view(-1)
            l = loss(output, y.long())
            # print('Y:\n',Y.shape,'\ny:\n',y.shape,'\noutput:\n',output.shape)
            
            optimizer.zero_grad()
            l.backward()
            # 梯度裁剪
            d2l.grad_clipping(model.parameters(), clipping_theta, device)
            optimizer.step()
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]
        
        try:
            perplexity = math.exp(l_sum / n)
        except OverflowError:
            perplexity = float('inf')
        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, perplexity, time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn_pytorch(
                    prefix, pred_len, model, vocab_size, device, idx_to_char,
                    char_to_idx))

In [26]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
output.backward()
print(input,target)
print(output)

tensor([[ 1.2985,  0.0281,  0.4858, -0.3609, -1.3564],
        [ 2.0052,  0.1069, -0.7606, -0.2188, -0.2630],
        [ 1.6754,  0.6015, -1.4906,  0.8020, -0.8566]], requires_grad=True) tensor([0, 3, 4])
tensor(2.1424, grad_fn=<NllLossBackward>)


In [27]:
y = torch.rand(3,2)
print(y,y.view((2,-1)))

tensor([[0.6343, 0.1975],
        [0.0139, 0.9474],
        [0.0448, 0.5758]]) tensor([[0.6343, 0.1975, 0.0139],
        [0.9474, 0.0448, 0.5758]])


In [29]:
num_epochs, batch_size, lr, clipping_theta = 500, 32, 1e-3, 1e-2 # 注意这里的学习率设置
pred_period, pred_len, prefixes = 50, 50, ['下雨', '天晴','阳光下']
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                            corpus_indices, idx_to_char, char_to_idx,
                            num_epochs, num_steps, lr, clipping_theta,
                            batch_size, pred_period, pred_len, prefixes)

epoch 50, perplexity 1.006534, time 0.04 sec
 - 下雨 的铁盒里藏著一片玫瑰花瓣 对不起 广场一枚铜币 悲伤得很隐密 它在许愿池里轻轻叹息 太多的我爱你 
 - 天晴天 所有回忆对着我进攻       所有回忆对着我进攻       古巴比伦王颁布了汉摩拉比法典 刻
 - 阳光下不再想你 爱你看棒球 想这样没担忧 唱着歌 一直走 我想就这样牵着你的手不放开 爱可不可以简简单单没
epoch 100, perplexity 1.005993, time 0.04 sec
 - 下雨 的为我较细汉 从小到大只有妈妈的温暖  为什么我爸爸 那么凶 如果真的我有一双翅膀 二双翅膀 随时
 - 天晴天 所有回忆对着我进攻       所有回忆对着我进攻       古巴比伦王颁布了汉摩拉比法典 刻
 - 阳光下不再想 我不能再想 我不 我不 我不能 爱情走的太快就像龙卷风 不能承受我已无处可躲 我不要再想 我
epoch 150, perplexity 1.005510, time 0.04 sec
 - 下雨 的为我较细汉 从小到大只有妈妈的温暖  为什么我爸爸 那么凶 如果真的我有一双翅膀 二双翅膀 随时
 - 天晴天 所有回忆对着我进攻       所有回忆对着我进攻       古巴比伦王颁布了汉摩拉比法典 刻
 - 阳光下不再想 我不要再想你我爱你开不知不觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋 
epoch 200, perplexity 1.006524, time 0.04 sec
 - 下雨 以为 较细汉 从小到大只有妈妈的温暖  为什么我爸爸 那么凶 如果真的我有一双翅膀 二双翅膀 随时
 - 天晴天 所有回忆对着我进攻       所有回忆对着我进攻       古巴比伦王颁布了汉摩拉比法典 刻
 - 阳光下不会气 我有妈妈 我有没有你看棒球 想这样没担忧 唱着歌 一直走 我想就这样牵着你的手不放开 爱可不
epoch 250, perplexity 1.005629, time 0.04 sec
 - 下雨 的布敌对野依旧每日折一枝杨柳 在在感受 河边的风 在吹着头发飘动 牵着你的手 一阵莫名感动 我想带
 - 天晴天 所有回忆对着我进攻 我的伤口被你拆封 誓言太沉重泪被纵容 脸上汹